In [18]:
import requests
from bs4 import BeautifulSoup
import csv

In [19]:
base_url = "https://www.amazon.in/s"
search_params = {
    "k": "bags",
    "crid": "2M096C61O4MLT",
    "qid": "1653308124",
    "sprefix": "ba%2Caps%2C283",
    "ref": "sr_pg_1"
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"  # Add your user agent
}
product_data = []

In [23]:
num_pages_to_scrape = 20
for page_num in range(1, num_pages_to_scrape + 1):
    search_params["page"] = page_num
    response = requests.get(base_url, headers=headers, params=search_params)
    soup = BeautifulSoup(response.content, "html.parser")

    products = soup.find_all("div", class_="s-result-item")
    for product in products:
        product_link = product.find("a", class_="a-link-normal")
        if product_link:
            product_url = "https://www.amazon.in" + product_link.get("href")
            product_name_elem = product_link.find("span", class_="a-text-normal")
            if product_name_elem:
                product_name = product_name_elem.get_text(strip=True)
            else:
                product_name = "N/A"
            
        else:
            product_url = "N/A"
            product_name = "N/A"
            
        product_price_elem = product.find("span", class_="a-offscreen")
        product_price = product_price_elem.get_text() if product_price_elem else "N/A"

        rating_elem = product.find("span", class_="a-icon-alt")
        rating = rating_elem.get_text().split()[0] if rating_elem else "N/A"

        num_reviews_elem = product.find("span", class_="a-size-base")
        num_reviews = num_reviews_elem.get_text() if num_reviews_elem else "N/A"

        # Append data to the list
        product_data.append([product_url, product_name, product_price, rating, num_reviews])


AttributeError: 'list' object has no attribute 'head'

In [28]:
for item in product_data:
    product_url = item[0]
    try:
        response = requests.get(product_url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract additional details
        description_element = soup.find("meta", {"name": "description"})
        if description_element:
            description = description_element.get("content")
        else:
            description = "N/A"

        asin_element = soup.find("th", text="ASIN")
        if asin_element:
            asin = asin_element.find_next("td").get_text()
        else:
            asin = "N/A"

        manufacturer_element = soup.find("th", text="Manufacturer")
        if manufacturer_element:
            manufacturer = manufacturer_element.find_next("td").get_text()
        else:
            manufacturer = "N/A"

        product_description_element = soup.find("div", id="productDescription")
        if product_description_element:
            product_description = product_description_element.get_text()
        else:
            product_description = "N/A"

        # Update the item's data
        item.extend([description, asin, product_description, manufacturer])
    except requests.exceptions.RequestException as e:
        print(f"Error for {product_url}: {e}")
with open("amazon_product_data.csv", "w", newline="", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Product URL", "Product Name", "Product Price", "Rating", "Number of Reviews", 
                         "Description", "ASIN", "Product Description", "Manufacturer"])
    
    for item in product_data:
        csv_writer.writerow(item)


Error for https://www.amazon.inhttps://aax-eu.amazon.in/x/c/RHJKfTDtCJ_12oizT9jLiOcAAAGKGMJ5eAMAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICCKV9A4/https://www.amazon.in/stores/page/BCF7AA48-B792-49F6-898D-ADEB25ED5FB6/?_encoding=UTF8&store_ref=SB_A02461542TLCACOY1JF7G&pd_rd_plhdr=t&aaxitk=532b18fd49c2772424529cf6898e459d&hsa_cr_id=6408141620802&lp_asins=B097L4GWB3%2CB07FZN5K3N&lp_query=bags&lp_slot=auto-sparkle-hsa-tetris&ref_=sbx_be_s_sparkle_lsi4d_bkgd&pd_rd_w=lebxO&content-id=amzn1.sym.bc0e77cc-908d-4121-87d3-b355b997dda7%3Aamzn1.sym.bc0e77cc-908d-4121-87d3-b355b997dda7&pf_rd_p=bc0e77cc-908d-4121-87d3-b355b997dda7&pf_rd_r=B4XFT0W6MR48N80N6EHJ&pd_rd_wg=ls1YR&pd_rd_r=2fc8e3a4-f212-4f27-9422-e9c5157ff5db: HTTPSConnectionPool(host='www.amazon.inhttps', port=443): Max retries exceeded with url: //aax-eu.amazon.in/x/c/RHJKfTDtCJ_12oizT9jLiOcAAAGKGMJ5eAMAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICCKV9A4/https://www.amazon.in/stores/page/BCF7AA48-B792-49F6-898D-ADEB25ED5FB6/?_encoding=

C:\Users\nitin\AppData\Local\Temp\ipykernel_1588\1472124435.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  asin_element = soup.find("th", text="ASIN")
C:\Users\nitin\AppData\Local\Temp\ipykernel_1588\1472124435.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  manufacturer_element = soup.find("th", text="Manufacturer")


Error for N/A: Invalid URL 'N/A': No scheme supplied. Perhaps you meant https://N/A?
Error for N/A: Invalid URL 'N/A': No scheme supplied. Perhaps you meant https://N/A?
Error for https://www.amazon.inhttps://aax-eu.amazon.in/x/c/RCi7a9MODipO_kLrzrex44cAAAGKGMJ5dAMAAAH2AQBvbm9fdHhuX2JpZDMgICBvbm9fdHhuX2ltcDEgICDMvoYK/https://www.amazon.in/dp/B09QL66G55?pd_rd_i=B09QL66G55&pf_rd_p=880c6186-3832-495e-9caa-7ecfb2f5e88c&pf_rd_r=B4XFT0W6MR48N80N6EHJ&pd_rd_wg=bZe7t&pd_rd_w=jzqOU&pd_rd_r=4796cb79-f44c-403b-ad95-198afda4083f: HTTPSConnectionPool(host='www.amazon.inhttps', port=443): Max retries exceeded with url: //aax-eu.amazon.in/x/c/RCi7a9MODipO_kLrzrex44cAAAGKGMJ5dAMAAAH2AQBvbm9fdHhuX2JpZDMgICBvbm9fdHhuX2ltcDEgICDMvoYK/https://www.amazon.in/dp/B09QL66G55?pd_rd_i=B09QL66G55&pf_rd_p=880c6186-3832-495e-9caa-7ecfb2f5e88c&pf_rd_r=B4XFT0W6MR48N80N6EHJ&pd_rd_wg=bZe7t&pd_rd_w=jzqOU&pd_rd_r=4796cb79-f44c-403b-ad95-198afda4083f (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object

Error for N/A: Invalid URL 'N/A': No scheme supplied. Perhaps you meant https://N/A?
Error for https://www.amazon.inhttps://aax-eu.amazon.in/x/c/RB6LggGCUuQD8N-2dO5VIDcAAAGKGMKMTgMAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICA1yNO-/https://www.amazon.in/stores/page/BCF7AA48-B792-49F6-898D-ADEB25ED5FB6/?_encoding=UTF8&store_ref=SB_A02461542TLCACOY1JF7G&pd_rd_plhdr=t&aaxitk=532b18fd49c2772424529cf6898e459d&hsa_cr_id=6408141620802&lp_asins=B097L4GWB3%2CB07FZN5K3N&lp_query=bags&lp_slot=auto-sparkle-hsa-tetris&ref_=sbx_be_s_sparkle_lsi4d_bkgd&pd_rd_w=Z0zDh&content-id=amzn1.sym.bc0e77cc-908d-4121-87d3-b355b997dda7%3Aamzn1.sym.bc0e77cc-908d-4121-87d3-b355b997dda7&pf_rd_p=bc0e77cc-908d-4121-87d3-b355b997dda7&pf_rd_r=FD4ZBKC6QN9X6QAV3SQV&pd_rd_wg=9TJbt&pd_rd_r=70db6e4a-339e-4ed3-ad51-a581b36d0e1d: HTTPSConnectionPool(host='www.amazon.inhttps', port=443): Max retries exceeded with url: //aax-eu.amazon.in/x/c/RB6LggGCUuQD8N-2dO5VIDcAAAGKGMKMTgMAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICA1yN

Error for N/A: Invalid URL 'N/A': No scheme supplied. Perhaps you meant https://N/A?
Error for https://www.amazon.inhttps://aax-eu.amazon.in/x/c/RLnVmjE-sjXy6ST6mOb-JC0AAAGKGMKiVQMAAAH2AQBvbm9fdHhuX2JpZDMgICBvbm9fdHhuX2ltcDEgICBJzUvq/https://www.amazon.in/stores/page/423336D6-E5A2-4A82-94A4-1BFE27DB8491/?_encoding=UTF8&store_ref=SB_A09138173D9TZ2WIMTWBE&pd_rd_plhdr=t&aaxitk=a16b04afae11576bf3b61a16b31e4e4f&hsa_cr_id=0&lp_asins=B07QHLBBPM&lp_query=bags&lp_slot=desktop-hsa-3psl&ref_=sbx_be_s_3psl_mbd_mb0_bkgd&pd_rd_w=xnTf6&content-id=amzn1.sym.f1a4d09d-a292-48da-a86f-03294c9a41ef%3Aamzn1.sym.f1a4d09d-a292-48da-a86f-03294c9a41ef&pf_rd_p=f1a4d09d-a292-48da-a86f-03294c9a41ef&pf_rd_r=D52S7DXNMZNXPK10MVC1&pd_rd_wg=RYxCl&pd_rd_r=6041cd89-3f11-4064-8498-a5bc66f206ae: HTTPSConnectionPool(host='www.amazon.inhttps', port=443): Max retries exceeded with url: //aax-eu.amazon.in/x/c/RLnVmjE-sjXy6ST6mOb-JC0AAAGKGMKiVQMAAAH2AQBvbm9fdHhuX2JpZDMgICBvbm9fdHhuX2ltcDEgICBJzUvq/https://www.amazon.in/stores/p

In [24]:
product_data


[['https://www.amazon.inhttps://aax-eu.amazon.in/x/c/RHJKfTDtCJ_12oizT9jLiOcAAAGKGMJ5eAMAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICCKV9A4/https://www.amazon.in/stores/page/BCF7AA48-B792-49F6-898D-ADEB25ED5FB6/?_encoding=UTF8&store_ref=SB_A02461542TLCACOY1JF7G&pd_rd_plhdr=t&aaxitk=532b18fd49c2772424529cf6898e459d&hsa_cr_id=6408141620802&lp_asins=B097L4GWB3%2CB07FZN5K3N&lp_query=bags&lp_slot=auto-sparkle-hsa-tetris&ref_=sbx_be_s_sparkle_lsi4d_bkgd&pd_rd_w=lebxO&content-id=amzn1.sym.bc0e77cc-908d-4121-87d3-b355b997dda7%3Aamzn1.sym.bc0e77cc-908d-4121-87d3-b355b997dda7&pf_rd_p=bc0e77cc-908d-4121-87d3-b355b997dda7&pf_rd_r=B4XFT0W6MR48N80N6EHJ&pd_rd_wg=ls1YR&pd_rd_r=2fc8e3a4-f212-4f27-9422-e9c5157ff5db',
  'N/A',
  '₹2,009.00',
  'N/A',
  'N/A'],
 ['N/A',
  'N/A',
  'N/A',
  'N/A',
  'Price and other details may vary based on product size and colour.'],
 ['https://www.amazon.in/Safari-Casual-Backpack-Charging-Resistant/dp/B09B2581S9/ref=sr_1_65?crid=2M096C61O4MLT&keywords=bags&qid=1692632